In [1]:
import gdown
import pandas as pd

# Define the URL of the CSV file
file_id = '1WClX4TtAVny-bz7AI1VNrqeGlnzyXiam'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file using gdown
output = 'sampled_data_full.csv'
gdown.download(url, output, quiet=False)

# Read the CSV file into a DataFrame
data = pd.read_csv(output)

# Display the last few rows of the DataFrame
data.tail()

Downloading...
From (original): https://drive.google.com/uc?id=1WClX4TtAVny-bz7AI1VNrqeGlnzyXiam
From (redirected): https://drive.google.com/uc?id=1WClX4TtAVny-bz7AI1VNrqeGlnzyXiam&confirm=t&uuid=c6326d1d-5973-4d71-9279-8ec77bcf5634
To: /Users/jinluwang/Downloads/Do work here/Challenges/Flight Delay/predictive_flight_delays_5/notebooks/sampled_data_full.csv
100%|██████████████████████████████████████████████| 268M/268M [00:09<00:00, 27.4MB/s]


,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
1297807,6,5,0,0800-0859,7,1,37,150,JetBlue Airways,12427,...,8,Logan International,42.364,-71.006,NONE,0.0,0.0,0.0,77.0,14.32
1297808,10,3,0,1100-1159,4,3,5,76,SkyWest Airlines Inc.,5961,...,0,Washington Dulles International,38.942,-77.458,Detroit Metro Wayne County,0.0,0.0,0.0,96.0,6.04
1297809,5,3,0,1900-1959,2,5,9,70,Comair Inc.,2558,...,15,Bradley International,41.939,-72.683,Ronald Reagan Washington National,0.0,0.0,0.0,68.0,6.93
1297810,6,3,1,0700-0759,4,2,45,76,SkyWest Airlines Inc.,21878,...,4,Stapleton International,39.774,-104.880,Billings Logan International,0.0,0.0,0.0,83.0,7.83
1297811,6,2,0,0700-0759,3,1,13,173,United Air Lines Inc.,4615,...,6,Kansas City International,39.298,-94.714,NONE,0.0,0.0,0.0,84.0,9.40


In [5]:
#highly compressed RF doesn't work

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Define feature matrix X and target vector y
features = ['MONTH', 'DAY_OF_WEEK', 'TMAX', 'AWND', 'AIRPORT_FLIGHTS_MONTH', 
            'AIRLINE_FLIGHTS_MONTH', 'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'PRCP', 'DEP_TIME_BLK']
target = 'DEP_DEL15'

X = data[features]
y = data[target]

# Encoding categorical features if necessary
label_encoders = {}
for feature in ['DEP_TIME_BLK']:  # Add more categorical features if any
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature])
    label_encoders[feature] = le

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Save the scaler and encoders
joblib.dump(scaler, '../trained_modules_R/standard_scaler.pkl')
joblib.dump(label_encoders, '../trained_modules_R/label_encoder.pkl')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model with reduced complexity
rf_model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)  # Reduced complexity
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the trained model with more aggressive compression
joblib.dump(rf_model, '../trained_modules_R/random_forest_model.pkl', compress=9)


/var/folders/h6/rstm4b2j0g928d95j1p6tr400000gn/T/ipykernel_52538/3129844574.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = le.fit_transform(X[feature])


Accuracy: 0.810735736603445


/Users/jinluwang/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jinluwang/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jinluwang/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Classification Report:
               precision    recall  f1-score   support

           0       0.81      1.00      0.90    210437
           1       0.00      0.00      0.00     49126

    accuracy                           0.81    259563
   macro avg       0.41      0.50      0.45    259563
weighted avg       0.66      0.81      0.73    259563



['../trained_modules_R/random_forest_model.pkl']

The below is the original RF model that will produce better result

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, accuracy_score
# import joblib


# # Define feature matrix X and target vector y
# features = ['MONTH', 'DAY_OF_WEEK', 'TMAX', 'AWND', 'AIRPORT_FLIGHTS_MONTH', 
#             'AIRLINE_FLIGHTS_MONTH', 'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'PRCP', 'DEP_TIME_BLK']
# target = 'DEP_DEL15'

# X = data[features]
# y = data[target]

# # Encoding categorical features if necessary
# label_encoders = {}
# for feature in ['DEP_TIME_BLK']:  # Add more categorical features if any
#     le = LabelEncoder()
#     X[feature] = le.fit_transform(X[feature])
#     label_encoders[feature] = le

# # Standardize the features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # Save the scaler and encoders
# joblib.dump(scaler, '../trained_modules_R/standard_scaler.pkl')
# joblib.dump(label_encoders, '../trained_modules_R/label_encoder.pkl')

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize and train the Random Forest model
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

# # Evaluate the model
# y_pred = rf_model.predict(X_test)
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

# # Save the trained model
# joblib.dump(rf_model, '../trained_modules_R/random_forest_model.pkl')

In [3]:
# from sklearn.metrics import confusion_matrix, classification_report
# # Print confusion matrix and classification report
# conf_matrix = confusion_matrix(y_test, y_pred)
# class_report = classification_report(y_test, y_pred)

# print('\nConfusion Matrix:')
# print(conf_matrix)

# print('\nClassification Report:')
# print(class_report)


Confusion Matrix:
[[194216  16221]
 [ 37565  11561]]

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.92      0.88    210437
           1       0.42      0.24      0.30     49126

    accuracy                           0.79    259563
   macro avg       0.63      0.58      0.59    259563
weighted avg       0.76      0.79      0.77    259563

